In [2]:
# from datetime import datetime
import pandas as pd

melbourne_file_path = 'melb_data.csv/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)
df = melbourne_data.describe()
df
# Самый старый и самый новый дом
# print(df['YearBuilt']['min'], int(datetime.now().year - df['YearBuilt']['min']), (datetime.now().year - df['YearBuilt']['max']))
# melbourne_data.columns


,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [3]:
melbourne_data.dropna(axis=0)  # Удаляем пропущенные значения
# Данные, которые хотим предсказать кладем в y
y = melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']  # фичи, влияющие на предсказания
X = melbourne_data[melbourne_features]
X.describe()
X.head(10)  # возвращает первые n строк массива данных


,Rooms,Bathroom,Landsize,Lattitude,Longtitude
0,2,1.0,202.0,-37.7996,144.9984
1,2,1.0,156.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,144.9944
3,3,2.0,94.0,-37.7969,144.9969
4,4,1.0,120.0,-37.8072,144.9941
5,2,1.0,181.0,-37.8041,144.9953
6,3,2.0,245.0,-37.8024,144.9993
7,2,1.0,256.0,-37.8060,144.9954
8,1,1.0,0.0,-37.8008,144.9973
9,2,1.0,220.0,-37.8010,144.9989


In [4]:
from sklearn.tree import DecisionTreeRegressor

melbourne_model = DecisionTreeRegressor(random_state=1)
melbourne_model.fit(X, y)
print("Making predictions for the following 5 houses:")
print(X.head())
print("The predictions are")
print(melbourne_model.predict(X.head()))

Making predictions for the following 5 houses:
   Rooms  Bathroom  Landsize  Lattitude  Longtitude
0      2       1.0     202.0   -37.7996    144.9984
1      2       1.0     156.0   -37.8079    144.9934
2      3       2.0     134.0   -37.8093    144.9944
3      3       2.0      94.0   -37.7969    144.9969
4      4       1.0     120.0   -37.8072    144.9941
The predictions are
[1480000. 1035000. 1465000.  850000. 1600000.]


In [5]:
from sklearn.metrics import mean_absolute_error
# Оценка качества модели

predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)


1125.1804614629357

In [6]:
from sklearn.model_selection import train_test_split
# разделение на обучающие и проверочные данные
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

melbourne_model = DecisionTreeRegressor()
melbourne_model.fit(train_X, train_y)

val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

248871.281688758


In [7]:
"""
Переобучение - слишком много листов, ложные закономерности, которые в будущем не повторяться, менее точный прогноз
Недообучение - мало листов, неспособность выявить закономерности, менее точный прогноз
"""
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    return mean_absolute_error(val_y, preds_val)

max_leaf_node_mae = {}
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    max_leaf_node_mae.update({max_leaf_nodes: my_mae})

best_leaf_cnt = min(max_leaf_node_mae, key=max_leaf_node_mae.get)

melbourne_model = DecisionTreeRegressor(max_leaf_nodes=best_leaf_cnt, random_state=0)
melbourne_model.fit(X, y)


DecisionTreeRegressor(max_leaf_nodes=500, random_state=0)

In [8]:
""" Случайный лес """
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
melb_pred = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_pred))


180860.37877504269
